# Hierarchical radon model

(c) 2019 Thomas Wiecki, Junpeng Lao

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pymc4 as pm 
import pandas as pd
import tensorflow as tf
import arviz as az

data = pd.read_csv('radon.csv')

county_names = data.county.unique()
county_idx = data['county_code'].values.astype(np.int32)

FileNotFoundError: [Errno 2] File b'/home/junpenglao/Documents/pymc3/docs/source/data/radon.csv' does not exist: b'/home/junpenglao/Documents/pymc3/docs/source/data/radon.csv'

In [ ]:
@pm.model
def hierarchical_model(data, county_idx):
    # Hyperpriors
    mu_a = yield pm.Normal('mu_alpha', mu=0., sigma=1)
    sigma_a = yield pm.HalfCauchy('sigma_alpha', beta=1)
    mu_b = yield pm.Normal('mu_beta', mu=0., sigma=1)
    sigma_b = yield pm.HalfCauchy('sigma_beta', beta=1)

    # Intercept for each county, distributed around group mean mu_a
    a = yield pm.Normal('alpha', mu=mu_a, sigma=sigma_a, plate=len(data.county.unique()))
    # Intercept for each county, distributed around group mean mu_a
    b = yield pm.Normal('beta', mu=mu_b, sigma=sigma_b, plate=len(data.county.unique()))

    # Model error
    eps = yield pm.HalfCauchy('eps', beta=1)

    # Expected value
    #radon_est = a[county_idx] + b[county_idx] * data.floor.values
    radon_est = tf.gather(a, county_idx) + tf.gather(
        b, county_idx) * data.floor.values

    # Data likelihood
    y_like = yield pm.Normal('y_like', mu=radon_est, sigma=eps, observed=data.log_radon)

In [ ]:
%%time
init_num_chains = 50
model = hierarchical_model(data, county_idx)
pm4_trace, _ = pm.inference.sampling.sample(
    model, num_chains=init_num_chains, num_samples=10, burn_in=10, step_size=1., xla=False)

### Mass matrix adaptation
Essentially, we are doing a 2 window adaptation similar to Stan below: dual averaging for burn_in samples, and then normal sample for diagonal mass matrix estimation.

In [ ]:
%%time
for i in range(3):
    step_size_ = []
    for _, x in pm4_trace.items():
        std = tf.math.reduce_std(x, axis=[0, 1])
        step_size_.append(
            std[tf.newaxis, ...] * tf.ones([init_num_chains] + std.shape, dtype=std.dtype))
    pm4_trace, _ = pm.inference.sampling.sample(
        model, num_chains=init_num_chains, num_samples=10 + 10*i, burn_in=10 + 10*i,
        step_size=step_size_, xla=False)

In [ ]:
%%time
num_chains = 5
step_size_ = []
for _, x in pm4_trace.items():
    std = tf.math.reduce_std(x, axis=[0, 1])
    step_size_.append(
        std[tf.newaxis, ...] * tf.ones([num_chains]+std.shape, dtype=std.dtype))

pm4_trace, sample_stat = pm.inference.sampling.sample(
    model, num_chains=num_chains, num_samples=500, burn_in=500,
    step_size=step_size_, xla=False)

In [ ]:
az_trace = pm.inference.utils.trace_to_arviz(pm4_trace, sample_stat)

In [ ]:
np.mean(np.exp(az_trace.sample_stats['mean_tree_accept']), axis=1)

In [ ]:
plt.hist(np.ravel(az_trace.sample_stats['tree_size']), 50);

In [ ]:
az.summary(az_trace)

In [ ]:
az.plot_energy(az_trace);

In [ ]:
az.plot_trace(az_trace, compact=True, combined=True);